In [175]:
import pandas as pd
from midiutil import MIDIFile

Let's import the files from net_data.

In [195]:
file_name='net_data/phone_on_4g.csv'

Put the CSV data into a Pandas data frame. Once it is in there, we will add columns

* Channel  - the stream for the connection.

* Time - the time of the event. Converted into epoch time by tshark

* Source  - the IP address that initiated the packet

* Target - the IP address that received the packet

* Data - the amount of data (in bytes) in the packet

In [196]:
phone_df = pd.read_csv(file_name, header=None)
phone_df.columns = ['Channel', 'Time', 'Source', 'Target', 'Data']
phone_df['Channel'] = phone_df['Channel'].replace('NaN', 0.0)
phone_df.head()

,Channel,Time,Source,Target,Data
0,0.0,1612538546,10.8.0.1,52.40.148.33,NaN
1,0.0,1612538546,52.40.148.33,10.8.0.1,NaN
2,1.0,1612538546,10.8.0.1,192.229.233.50,NaN
3,1.0,1612538546,192.229.233.50,10.8.0.1,NaN
4,0.0,1612538546,10.8.0.1,52.40.148.33,NaN


Get the first time stamp so that we have a 0 point in time to calculate from

In [197]:
start = phone_df['Time'].min()

Set the values for the MIDI variables

In [198]:
track    = 0
channel  = 0
time     = 0    # In beats
duration = 1    # In beats
tempo    = 60   # In BPM
volume   = 62  # 0-127, as per the MIDI standard

In [199]:
# One track, defaults to format 1 (tempo track is created automatically)
# set tracks to max number of streams connected
num_tracks = int(phone_df['Channel'].max())
MyMIDI = MIDIFile(num_tracks)  
MyMIDI.addTempo(track, time, tempo)

We can't put an IP address here if we want to focus on a particular IP address. 

In [153]:
filtered_ip = 10

Let's iterate over the rows in the dataframe and filter by source or target. If the ip is being filtered, then the volume is louder than non-filtered ips. These are played at the time that they appear in the data stream. 

In [154]:
import math

for _, row in phone_df.iterrows():
    if type(row['Source']) is not float and type(row['Target']):

        #if not math.isnan(row['Data']):
        #    d = int(row['Data'])
        #    if d < 1000:
        #        duration = d/10000
        #    elif d < 100:
        #        duration = d/1000
        #    else:
        #        duration = d/100
                
        #    print(int(row['Data'])/100)
        #    print(type(row['Data']))
        
        source = int(row['Source'].split('.')[0])
        target = int(row['Target'].split('.')[0])
        _time = int(row['Time']-start)
 
        if type(row['Channel']) is not float:
            channel = row['Channel']
        else:
            channel = 0
            
        if source > 127: source = int(source/2)
        if target > 127: target = int(target/2)

        #source == filtered_ip
        if source not in [10, 192,46,129]:
            MyMIDI.addNote(channel, channel, source, _time, 1, 90)
        else:
            MyMIDI.addNote(channel, channel, source, _time, 1, 60)
        #create a note for the source

        #create a slight offset note for the target
        targettime = int(_time + 0.1)

        if target not in [10, 192,46,129]:
            MyMIDI.addNote(channel, channel, target, targettime, 1, 90)
        else:
            MyMIDI.addNote(channel, channel, target, targettime, 1, 60)
            
with open(file_name[:-4]+"filter.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

Let's iterate over the data frame. Here we use the time difference between the time that the source/target pair have been seen. If they have been seen recently, then make it louder, If it is quieter, then it was seen a while ago. 

In [ ]:
import math
#let's set up a simple dictionary to hold the ips and the time for a simple calculation
ips = {}

for _, row in phone_df.iterrows():
    if type(row['Source']) is not float and type(row['Target']):
        

        #if not math.isnan(row['Data']):
        #    print(int(row['Data'])/100)
        #    print(type(row['Data']))
            
        source = row['Source']
        target = row['Target']
        
        if type(row['Channel']) is not float:
            channel = row['Channel']
        else:
            channel = 0
        
        #Have we seen it before?
        if source+target not in ips:
            ips[source+target] = {'time':0,'volume':60}
        else:
            #get the current last time. 
            tmp = ips[source+target]["time"]
            
            time_diff = (int(row['Time']) - ips[source+target]["time"])
            #if closer, then increase the volume
            if time_diff > ips[source+target]["time"]:
                if ips[source+target]["volume"] < 127:
                    ips[source+target]["volume"] += 1
            #if further away, then decrease the volume
            elif time_diff < ips[source+target]["time"]:
                if ips[source+target]["volume"] > 0:
                    ips[source+target]["volume"] -= 1
            
            ips[source+target] = {'time':time_diff,'volume':volume} 
  
        _time = int(row['Time']-start)

        if int(source.split('.')[0]) > 127:
            _s = 80
        else:
            _s = int(source.split('.')[0])
            
        if int(target.split('.')[0]) > 127:
            _t = 75
        else:
            _t = int(target.split('.')[0])     

        #create a note for the source
        MyMIDI.addNote(channel, channel, _s, _time, 1, volume)
        #create a slight offset note for the target
        
        targettime = int(_time + 1)
        MyMIDI.addNote(channel, channel, _t, targettime, 1, volume)

with open(file_name[:-4]+"ts.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

Let's map to the underlying companies in the connections. 

In [203]:
import math

for _, row in phone_df.iterrows():
    if type(row['Source']) is not float and type(row['Target']):

        duration = 1
        if not math.isnan(row['Data']):
            d = int(row['Data'])
            if d < 1000:
                duration = d/1000
            elif d < 100:
                duration = d/10
            else:
                duration = d
                
        #    print(int(row['Data'])/100)
        print(duration)
        
        source = int(row['Source'].split('.')[0])
        target = int(row['Target'].split('.')[0])
        _time = int(row['Time']-start)
        
        if source in [34,74,142,172,216] or target in [34,74,142,172,216]:
            #Google
            pitch = 80
        elif source == 192 or target == 192:
            #Verizon
            pitch = 90
        elif source == 52 or target == 52:
            #Microsoft
            pitch = 52            
        elif source == 17 or target == 17:
            #Apple
            pitch = 17
        elif source == 35 or target == 35:
            #AWS
            pitch = 35
        elif source == 46 or target == 46:
            #Hetzner
            pitch = 46
        else:
            #let's go for the default
            pitch = 15
            

        if type(row['Channel']) is not float:
            channel = row['Channel']
        else:
            channel = 0


        #source == filtered_ip
        if source < 16:
            MyMIDI.addNote(channel, channel, pitch, _time, duration, 90)
        else:
            MyMIDI.addNote(channel, channel, pitch, _time, duration, 60)
        #create a note for the source

        #create a slight offset note for the target
        targettime = int(_time + 0.1)

        if target < 16:
            MyMIDI.addNote(channel, channel, pitch, targettime, duration, 90)
        else:
            MyMIDI.addNote(channel, channel, pitch, targettime, duration, 60)
            
with open(file_name[:-4]+"platform.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

1
1
1
1
1
1
0.517
1
0.517
1
1
1
1
0.517
1
1
1
1
1
1
1
0.517
1
1
1
0.517
1
1
0.517
1
1
1
1
1
1
1
1
1
2885
4006
3401
1
1
4006
2885
3401
1
1
1
1
1
0.093
1
1
1
1
1
1
0.126
1
0.126
1
0.093
1
0.536
1
0.316
1
0.177
1
0.311
1
0.08
1
0.08
1
0.17
1
0.194
1
0.187
1
0.221
1
0.517
1
0.327
0.038
1
0.606
1
0.327
0.024
1
1
1
1
1
0.031
1
1
1
1
1
1
0.051
0.051
1
1
0.637
1
0.031
1
1208
1
1824
1
9852
1
1
1
1
0.533
1
0.064
1
0.213
1
0.3
1
0.27
1
0.536
1
0.536
1
0.043
1
0.031
1
1
1
1
1
1208
1
22952
0.103
1
1
2416
1
21744
1
4832
1
14496
1
1208
1
0.27
1208
1
1685
2416
0.536
1
0.203
0.263
1
1
1
1
0.064
1
7248
0.08
0.401
1
0.536
1
0.536
1
0.475
1
1
1
2416
1
9664
1
0.1
1
14496
1
38656
1
19328
1
1208
1
3624
1
1208
1
4832
1
3624
1
0.27
1
1208
1
3624
1
0.798
1
7248
1
1208
1
6040
1
1208
1
0.08
35032
1
1
1208
1
12080
1
6040
1
2416
1
3624
1
25368
0.1
1
1
4832
1
2416
1
15704
1
3624
1
1208
1
10872
1
4832
1
1208
1
6040
1
26576
1
1208
1
12080
1
7248
1
3624
1
2416
1
3624
1
4832
1
2416
1
14496
1
6040
1
0.647
8456
1
15704
1


4832
1
10872
1
9664
1
1208
1
7248
1
13288
1
9664
1
0.1
1
1208
1
15704
1
7248
1
1208
1
13288
1
2416
1
4832
1
13288
1
13288
1
7248
1
7248
1
7248
1
3624
1
1208
1
12080
1
9664
1
3624
1
0.08
6040
1
1
1208
1
10872
1
2416
1
7248
1
10872
1
8456
1
3624
1
4832
1
9664
1
8456
1
9664
1
8456
0.1
1
1
2416
1
12080
1
9664
1
9664
1
13288
1
6040
1
4832
1
15704
1
13288
1
14496
1
8456
1
12080
1
0.08
8456
1
1
3624
1
2416
1
1208
1
14496
1
1208
1
15704
1
12080
1
12080
1
1208
1
7248
1
10872
1
10872
1
3624
1
1208
1
1208
1
8456
1
6040
0.1
1
1
1208
1
12080
1
6040
1
7248
1
8456
1
8456
1
7248
1
19328
1
9664
1
7248
1
1208
1
9664
1
6040
1
8456
1
2416
1
4832
1
4832
1
0.08
14496
1
1
1208
1
2416
1
8456
1
13288
1
16912
1
15704
1
13288
1
16912
1
14496
1
19328
1
3624
1
9664
1
2416
1
24160
0.1
1
1
2416
1
15704
1
7248
1
16912
1
3624
1
1208
1
1208
1
4832
1
3624
1
1208
1
3624
1
8456
1
2416
1
1208
1
10872
1
3624
1
2416
1
12080
1
9664
1
6040
1
0.08
13288
1
1
1208
1
6040
1
2416
1
2416
1
3624
1
15704
1
26576
1
12080
0.1
1
1
2416
1

18120
1
41072
1
49528
1
1208
1
6040
1
16912
1
43488
1
35032
1
13288
1
42280
1
25368
1
44696
1
16912
0.1
1
1
1208
1
21744
1
21744
1
0.08
51944
1
1
7248
1
35032
1
49528
1
25368
1
2416
1
36240
1
1208
1
36240
0.1
1
1
1208
1
25368
0.042
1
0.12
1
1
1
1208
1
26576
1
0.08
42280
1
1
1208
1
32616
1
8456
1
45904
1
37448
1
37448
1
10872
1
7248
1
27784
1
42280
1
49528
1
26576
1
36240
1
45904
1
16912
1
30200
1
50736
1
10872
1
1208
1
25368
1
50736
1
51944
1
28992
1
33824
1
53152
1
10872
1
28992
1
42280
1
25368
1
41072
1
47112
1
24160
1
14496
1
51944
1
28992
1
21744
1
38656
1
54360
1
22952
1
27784
1
28992
1
47112
1
30200
1
35032
1
24160
0.042
1
1
1
1208
1
1208
1
24160
1
38656
1
42280
1
13288
1
2416
1
2416
1
1208
1
2416
1
37448
1
33824
1
38656
1
50736
1
7248
1
36240
1
39864
1
51944
1
16912
1
38656
1
55568
1
21744
1
42280
1
4832
1
36240
1
55568
1
26576
1
30200
1
44696
1
14496
1
2416
1
3624
1
1208
1
1208
1
1
1
1
0.517
1
1208
1
1208
1
2416
1
15704
1
41072
1
41072
1
37448
1
28992
1
2887
55568
1
1
8456
1
31

In [45]:
print(ips)

{'10.8.0.164.233.184.188': {'time': 1612538024, 'volume': 62}, '64.233.184.18810.8.0.1': {'time': 0, 'volume': 62}, '192.168.1.5172.217.169.4': {'time': 1612538025, 'volume': 62}, '172.217.169.4192.168.1.5': {'time': 1612538025, 'volume': 62}, '10.8.0.1192.168.1.1': {'time': 1612538025, 'volume': 62}, '192.168.1.110.8.0.1': {'time': 1612538025, 'volume': 62}, '10.8.0.1172.217.169.4': {'time': 1612538025, 'volume': 62}, '172.217.169.410.8.0.1': {'time': 0, 'volume': 62}, '10.8.0.1129.102.3.227': {'time': 1612538025, 'volume': 62}, '129.102.3.22710.8.0.1': {'time': 1612538145, 'volume': 62}, '10.8.0.146.4.77.182': {'time': 1612538147, 'volume': 62}, '46.4.77.18210.8.0.1': {'time': 1612538147, 'volume': 62}, '10.8.0.192.242.132.16': {'time': 0, 'volume': 62}, '92.242.132.1610.8.0.1': {'time': 1612538026, 'volume': 62}, '10.8.0.1216.58.212.197': {'time': 80, 'volume': 62}, '216.58.212.19710.8.0.1': {'time': 1612538097, 'volume': 62}, '10.8.0.1142.250.178.3': {'time': 1612538069, 'volume': 